# Pair Programming ANOVA

`En el pair programming de hoy usaremos el set de datos que guardastéis en el pair programming de normalización y estandarización.`

Hasta ahora habéis estado evaluando las características de vuestro set de datos y habéis hecho una gran exploración, es el momento de hacer vuestro primer ANOVA! En el ejercicio de hoy tendréis que hacer un ANOVA con vuestro datos y hacer una interpretación de los resultados.

📌 NOTA Puede que vuestros datos no se ajusten o no cumplan todas las asunciones, no pasa nada, haced el ANOVA e interpretad los resultados. En próximas lecciones aprenderemos que podemos hacer cuando nos encontramos en esta situación.



In [10]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import scipy.stats as stats
from scipy.stats import levene
import statsmodels.api as sm
from statsmodels.formula.api import ols
from statsmodels.multivariate.manova import MANOVA
from sklearn.preprocessing import StandardScaler

plt.rcParams["figure.figsize"] = (10,8) 

In [11]:
df = pd.read_csv('../Datos/vinos_norm_estandar.csv', index_col = 0)
df.head(2)

,id,year_birth,education,marital_status,income,kidhome,teenhome,dt_customer,recency,mntwines,...,mntsweetproducts,mntgoldprods,numdealspurchases,numwebpurchases,numcatalogpurchases,numstorepurchases,numwebvisitsmonth,mnt_total_sin_vino,mntwines_sin_cero,mnt_wines_box
0,5524,1957,Graduation,Single,0.235696,0,0,04-09-2012,0.307039,635,...,1.476500,0.843207,0.349414,1.409304,2.51089,-0.550785,0.693904,2.011116,635.0,14.026740
1,2174,1954,Graduation,Single,-0.235454,1,1,08-03-2014,-0.383664,11,...,-0.631503,-0.729006,-0.168236,-1.110409,-0.56872,-1.166125,-0.130463,-0.845274,11.0,3.141178


> Recordamos en la comprobación que hicimos de las asunciones para el ANOVA que:
> * No logramos alcanzar la normalidad de la variable respuesta
> * Varias de nuestras variables predictoras son dependientes entre sí
> * Solo la variable education es homocedástica en relación con la variable respuesta
>
> Sin embargo queremos probar de nuevo la homocedasticidad ahora que nuestra variable respueta está más normalizada.

In [12]:
#Realizamos el Test de Levene para las columnas numéricas
numericas= df.select_dtypes(np.number)

In [13]:
for col in numericas.columns:
    _, p_valor = stats.levene(df[col], df['mnt_wines_box'])
    print('homocedasticidad entre', col, 'y mnt_wines_box: p-valor = ', p_valor)

homocedasticidad entre id y mnt_wines_box: p-valor =  0.0
homocedasticidad entre year_birth y mnt_wines_box: p-valor =  1.4169744995337058e-239
homocedasticidad entre income y mnt_wines_box: p-valor =  0.0
homocedasticidad entre kidhome y mnt_wines_box: p-valor =  0.0
homocedasticidad entre teenhome y mnt_wines_box: p-valor =  0.0
homocedasticidad entre recency y mnt_wines_box: p-valor =  0.0
homocedasticidad entre mntwines y mnt_wines_box: p-valor =  0.0
homocedasticidad entre mntfruits y mnt_wines_box: p-valor =  0.0
homocedasticidad entre mntmeatproducts y mnt_wines_box: p-valor =  0.0
homocedasticidad entre mntfishproducts y mnt_wines_box: p-valor =  0.0
homocedasticidad entre mntsweetproducts y mnt_wines_box: p-valor =  0.0
homocedasticidad entre mntgoldprods y mnt_wines_box: p-valor =  0.0
homocedasticidad entre numdealspurchases y mnt_wines_box: p-valor =  0.0
homocedasticidad entre numwebpurchases y mnt_wines_box: p-valor =  0.0
homocedasticidad entre numcatalogpurchases y mnt_

Aclaramos de nueva cuenta que no tenemos homocedasticidad.
Sin embargo intentaremos con el test de ANOVA.

In [16]:
print(df.columns)

Index(['id', 'year_birth', 'education', 'marital_status', 'income', 'kidhome',
       'teenhome', 'dt_customer', 'recency', 'mntwines', 'mntfruits',
       'mntmeatproducts', 'mntfishproducts', 'mntsweetproducts',
       'mntgoldprods', 'numdealspurchases', 'numwebpurchases',
       'numcatalogpurchases', 'numstorepurchases', 'numwebvisitsmonth',
       'mnt_total_sin_vino', 'mntwines_sin_cero', 'mnt_wines_box'],
      dtype='object')


In [17]:
# iniciamos nuestro anova. Recordemos que la variable respuesta es el precio de las casas (median_house_value)
lm = ols('mnt_wines_box ~ year_birth  + education + marital_status + income + kidhome + teenhome + dt_customer + recency + mntfruits + mntmeatproducts + mntfishproducts + mntsweetproducts + mntgoldprods + numdealspurchases + numwebpurchases + numcatalogpurchases + numstorepurchases + numwebvisitsmonth + mnt_total_sin_vino', data=df).fit()
sm.stats.anova_lm(lm)

,df,sum_sq,mean_sq,F,PR(>F)
education,4.0,5594.935721,1398.733930,318.257451,5.189402e-200
marital_status,7.0,165.837893,23.691128,5.390502,4.040689e-06
dt_customer,662.0,18107.589270,27.352854,6.223664,5.365387e-192
year_birth,1.0,1165.218155,1165.218155,265.125019,3.755931e-55
income,1.0,12208.341117,12208.341117,2777.794579,0.000000e+00
kidhome,1.0,2255.664552,2255.664552,513.237032,2.144621e-98
teenhome,1.0,180.899071,180.899071,41.160421,1.857891e-10
recency,1.0,29.654139,29.654139,6.747281,9.477921e-03
mntfruits,1.0,684.504801,684.504801,155.747100,3.944139e-34
mntmeatproducts,1.0,752.183984,752.183984,171.146315,3.568987e-37


Rechazamos que el predictor no afecta a la variable respuesta, es decir, sí estarían afectando a nuestra variable respuesta.